In [ ]:
from __future__ import print_function

import os
from datetime import datetime, date, time, timezone
import numpy as np
import matplotlib.pyplot as plt
#import ipywidgets as widgets
#import IPython.display

import pyTMD.io
import pyTMD.time
import pyTMD.predict
import pyTMD.tools
import pyTMD.utilities

# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
model = pyTMD.io.model('C:\\Users\\sibekwut\\OneDrive - University of New Brunswick\\Fall 2022', format='FES',
            compressed=False).elevation('FES2014')

In [ ]:
date_str = ['2022-06-02 00:54:37.99', '2022-06-02 00:54:38.99', '2022-06-02 00:54:39.99', '2022-06-02 00:54:40.99', '2022-06-02 00:54:41.99']

LAT = [50.1376756306, 50.1377019083, 50.1377282889, 50.1377543472, 50.1377802667]

LON = [-60.7275792, -60.7275072222, -60.7274354028, -60.7273631944, -60.7272904444]

In [ ]:
#gnss_contents['Timestamp'].apply(lambda a: datetime.strptime(a, '%Y-%m-%d %H:%M:%S.%f').timestamp())
d = list(map(lambda i: np.datetime64(datetime.strptime(date_str[i], '%Y-%m-%d %H:%M:%S.%f')), range(len(date_str))))

tide_time = np.array(pyTMD.time.convert_datetime(d, epoch =pyTMD.time._tide_epoch)/86400)

# number of time points
nt = len(tide_time)

# delta time (TT - UT1) file
delta_file = pyTMD.utilities.get_data_path(['data','merged_deltat.data'])

tide_time

In [ ]:
if model.format == 'FES':
    constituents = pyTMD.io.FES.read_constants(model.model_file,
        type=model.type, version=model.version,
        compressed=model.compressed)
    c = model.constituents
    # interpolate delta times from calendar dates to tide time
    DELTAT = pyTMD.time.interpolate_delta_time(delta_file, tide_time)
    print(DELTAT)

In [ ]:
# update the tide prediction and plot
def update_tide_prediction():
    if model.format in ('FES'):
        amp,ph = pyTMD.io.FES.interpolate_constants(
            np.atleast_1d(LON), np.atleast_1d(LAT),
            constituents, scale=model.scale,
            method='spline', extrapolate=True)
    # calculate complex phase in radians for Euler's
    cph = -1j*ph*np.pi/180.0
    # calculate constituent oscillation
    hc = amp*np.exp(cph)
                
    # predict tidal elevations at time 1 and infer minor corrections
    TIDE = pyTMD.predict.drift(tide_time, hc, c,
        deltat=DELTAT, corrections=model.format)
    MINOR = pyTMD.predict.infer_minor(tide_time, hc, c,
        deltat=DELTAT, corrections=model.format)
    TIDE.data[:] += MINOR.data[:]
    # convert to meters
    TIDE.data[:] *= 1
    
    print('date=', date_str)
    print('Lon=', LON)
    print('Lat=', LAT)
    print('Tide=',TIDE)
    
    
# run tide prediction at initial location
update_tide_prediction()